# The following Python Notebook will allow the user to use a bounding box to retrieve the elevation of each point in an existing array within a bounding box.
[Link to StackOverflow question](https://stackoverflow.com/questions/26000336/execute-curl-command-within-a-python-script) on using HTTP requests in Python.
[Link to documentation](https://requests.readthedocs.io/en/latest/) on Python requests library.

# Basics of 2D arrays in Python
The following section is going to design basic 2d arrays for informational purposes. It's something I have never done before. So we test it first.

In [2]:
# Print a 2d Array in python. Two Methods.
import random
array = [[ random.randint(0,4) for x in range(0,4)] for y in range(0,4)]
print(array)

two_d_array = [[2,4,8],[1,5,8],[9,3,7]]
print(two_d_array)

[[1, 3, 0, 4], [4, 2, 2, 1], [1, 4, 0, 2], [0, 3, 4, 4]]
[[2, 4, 8], [1, 5, 8], [9, 3, 7]]


# Basics of Python Request Library (Running basic requests)
## This section is going to describe how to query the API to return the Elevation data for the surrounding area.

In [3]:
import requests

# Run a GET request
r = requests.get('https://api.github.com/events')

# Display the output in JSON format
r.json()

[{'id': '42838751153',
  'type': 'IssueCommentEvent',
  'actor': {'id': 6279541,
   'login': 'mplawley',
   'display_login': 'mplawley',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/mplawley',
   'avatar_url': 'https://avatars.githubusercontent.com/u/6279541?'},
  'repo': {'id': 288500503,
   'name': 'maustinstar/liquid',
   'url': 'https://api.github.com/repos/maustinstar/liquid'},
  'payload': {'action': 'created',
   'issue': {'url': 'https://api.github.com/repos/maustinstar/liquid/issues/12',
    'repository_url': 'https://api.github.com/repos/maustinstar/liquid',
    'labels_url': 'https://api.github.com/repos/maustinstar/liquid/issues/12/labels{/name}',
    'comments_url': 'https://api.github.com/repos/maustinstar/liquid/issues/12/comments',
    'events_url': 'https://api.github.com/repos/maustinstar/liquid/issues/12/events',
    'html_url': 'https://github.com/maustinstar/liquid/pull/12',
    'id': 2471839366,
    'node_id': 'PR_kwDOETIrF854p_7g',
    'number': 

## This section will run Googles test data for the API.
After researching the given location myself, it turns out this location is the Colorado State Capitol building, which rests at an elevation of about 1609 metres. Which is then returned by the API upon request.

In [ ]:
import requests
# Sample Given by Google API documentation. *Note, insert an API key to run this request. It has been removed for security reasons.
r = requests.get('https://maps.googleapis.com/maps/api/elevation/json?locations=39.7391536%2C-104.9847034&key=api_key')
r.json()
# Returns the elevation of the Denver Capitol Building.

## The following is the result of plugging the co-ordinates into Google maps for the summit of Carrauntoohil.
Using Google Maps, the summit of Carrauntoohil returns coordinates of 51.999445 for latitude, and -9.742693 for the latitude.
Using these coordinates, the returned result should roughly match Carrauntoohils reported summit height of 1038.6

In [1]:
import requests
# Note: The API code has been removed from this for security concerns. It would have to be reinserted to work again. It worked, you'll just have to take my word for it.
res = requests.get('https://maps.googleapis.com/maps/api/elevation/json?locations=51.999445%2C-9.742693&key=apikey')
res.json()
# This has been run twice. The one above has been run once
api_call_counter = 3

{'results': [{'elevation': 994.4034423828125,
   'location': {'lat': 51.999445, 'lng': -9.742693},
   'resolution': 152.7032318115234}],
 'status': 'OK'}

## The results from the previous Python code is the following:
```
{
    "results": [
        {
            "elevation": 994.4034423828125,
            "location": {
                "lat": 51.999445,
                "lng": -9.742693
            },
            "resolution": 152.7032318115234
        }
    ],
    "status": "OK"
}
```

Where the Elevation property returns 994 meters. This is close enough to the summit, although I thought it would have been closer, given this is near the Google maps area for the summit,
and it is what [Geohack](https://geohack.toolforge.org/geohack.php?pagename=Carrauntoohil&params=51.999445_N_9.742693_W_type:mountain_scale:100000) reports with their page on the summit.

## The next objective is to draw a distance from the summit of exactly 1 kilometre for the bounding box, then find the coordinates for the box.
Given that the Summit is at 51.999445, -9.742693 the task here is to calculate the coordinates that are 1 km away in any direction.

In [2]:
# The following block of code will design a 2d array of bounding box co-ordinates
bounding_box=[[52.00844, -9.7573],[52.00844,-9.728086],[51.99045,-9.7573],[51.99045,-9.728086]]
print(bounding_box)

max_latitude = bounding_box[0][0]
min_latitude = bounding_box[2][0]
latitude_difference = max_latitude - min_latitude
print(round(latitude_difference, 5)) # Answer is corrected to 5 decimal places to prevent rounding errors.

max_longitude = bounding_box[0][1]
min_longitude = bounding_box[1][1]
longitude_difference = max_longitude - min_longitude
print(round(longitude_difference, 6))

[[52.00844, -9.7573], [52.00844, -9.728086], [51.99045, -9.7573], [51.99045, -9.728086]]
0.01799
-0.029214


## Designing a for loop to make api calls with the required co-ordinates.

In [3]:
steps = 20 # Step counter has to go to 19 instead of 20 to prevent loop index errors. Previously, it went up to 21 each.

lat_diff = (max_latitude - min_latitude) / steps
long_diff = (max_longitude - min_longitude) / steps

counter = 0
for i in range(0,steps):
    for j in range (0,steps):
        counter += 1
        print("Coords (" + str(min_latitude + i * lat_diff) + " , " + str(min_longitude + j * long_diff) + ")")
print(counter) # Keeping a counter to divide out the number of API calls I am going to need. Possibly a Sleep function too to prevent too many API calls being made.

Coords (51.99045 , -9.728086)
Coords (51.99045 , -9.7295467)
Coords (51.99045 , -9.7310074)
Coords (51.99045 , -9.7324681)
Coords (51.99045 , -9.7339288)
Coords (51.99045 , -9.7353895)
Coords (51.99045 , -9.7368502)
Coords (51.99045 , -9.7383109)
Coords (51.99045 , -9.7397716)
Coords (51.99045 , -9.7412323)
Coords (51.99045 , -9.742693)
Coords (51.99045 , -9.7441537)
Coords (51.99045 , -9.745614400000001)
Coords (51.99045 , -9.7470751)
Coords (51.99045 , -9.7485358)
Coords (51.99045 , -9.7499965)
Coords (51.99045 , -9.7514572)
Coords (51.99045 , -9.7529179)
Coords (51.99045 , -9.7543786)
Coords (51.99045 , -9.7558393)
Coords (51.991349500000005 , -9.728086)
Coords (51.991349500000005 , -9.7295467)
Coords (51.991349500000005 , -9.7310074)
Coords (51.991349500000005 , -9.7324681)
Coords (51.991349500000005 , -9.7339288)
Coords (51.991349500000005 , -9.7353895)
Coords (51.991349500000005 , -9.7368502)
Coords (51.991349500000005 , -9.7383109)
Coords (51.991349500000005 , -9.7397716)
Coords

## Calculating the price of the Bounding Box
So with a Counter value at the end of the nested loop displaying 400 counts, which would break the search area into boxes of 20 meters squared.
